<a href="https://colab.research.google.com/github/Uday-ML/LS-2024-Machine-Learning/blob/main/LS_Assignment_2_Neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip homer_bart.zip

Archive:  homer_bart.zip
replace Bart/bart58.bmp? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import tensorflow as tf
import os
from google.colab import drive

# Mount Google Drive
#drive.mount('/content/drive')

# Paths
#base_dir = '/content/drive/My Drive/path_to_extracted_files'  # Update this to your extraction path
bart_dir = 'Bart'
homer_dir = os.path.join('Homer')

# Get the list of BMP image file paths from both folders
file_paths = []
for dir_path in [bart_dir, homer_dir]:
    file_paths += [os.path.join(dir_path, f) for f in os.listdir(dir_path) if f.endswith('.bmp')]

# Function to load and preprocess BMP images
def load_and_preprocess_image(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_bmp(image, channels=3)  # Decode BMP images
    image = tf.image.resize(image, [64, 64])
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]
    return image

# Create a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices(file_paths)
dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)


# Batch the dataset
batch_size = 32
dataset = dataset.batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# To iterate through the dataset
for batch in dataset:
    # Perform your operations on the batch
    print(batch.shape)


(32, 64, 64, 3)
(32, 64, 64, 3)
(32, 64, 64, 3)
(32, 64, 64, 3)
(32, 64, 64, 3)
(32, 64, 64, 3)
(32, 64, 64, 3)
(32, 64, 64, 3)
(13, 64, 64, 3)


In [ ]:
import tensorflow as tf
import os
import numpy as np
from sklearn.model_selection import train_test_split

bart_dir = os.path.join('Bart')
homer_dir = os.path.join('Homer')

# Get the list of BMP image file paths from both folders
file_paths_bart = [os.path.join(bart_dir, f) for f in os.listdir(bart_dir) if f.endswith('.bmp')]
file_paths_homer = [os.path.join(homer_dir, f) for f in os.listdir(homer_dir) if f.endswith('.bmp')]

# Load and preprocess images
def load_and_preprocess_image(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_bmp(image, channels=3)  # Decode BMP images
    image = tf.image.resize(image, [64, 64])
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]
    return image

# Load images and labels
images = []
labels = []
for file_path in file_paths_bart:
    images.append(load_and_preprocess_image(file_path))
    labels.append(0)  # Assuming 0 represents "Bart"
for file_path in file_paths_homer:
    images.append(load_and_preprocess_image(file_path))
    labels.append(1)  # Assuming 1 represents "Homer"

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Split data into training and test sets (9:1 ratio)
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.1, random_state=42)

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Dropout for regularization
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=20, batch_size=32, validation_split=0.0)  # No validation set, only training

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_accuracy}')


Epoch 1/20
8/8 [==============================] - 5s 327ms/step - loss: 0.6855 - accuracy: 0.5661
Epoch 2/20
8/8 [==============================] - 2s 192ms/step - loss: 0.6694 - accuracy: 0.6074
Epoch 3/20
8/8 [==============================] - 1s 184ms/step - loss: 0.6598 - accuracy: 0.6074
Epoch 4/20
8/8 [==============================] - 1s 182ms/step - loss: 0.6565 - accuracy: 0.6074
Epoch 5/20
8/8 [==============================] - 2s 192ms/step - loss: 0.6479 - accuracy: 0.6157
Epoch 6/20
8/8 [==============================] - 1s 182ms/step - loss: 0.6378 - accuracy: 0.6116
Epoch 7/20
8/8 [==============================] - 1s 185ms/step - loss: 0.6109 - accuracy: 0.6488
Epoch 8/20
8/8 [==============================] - 2s 191ms/step - loss: 0.5997 - accuracy: 0.6529
Epoch 9/20
8/8 [==============================] - 3s 320ms/step - loss: 0.5834 - accuracy: 0.6860
Epoch 10/20
8/8 [==============================] - 3s 317ms/step - loss: 0.5492 - accuracy: 0.7355
Epoch 11/20
8/8 [==